# Used Cars Analysis Pipeline - PySpark
## 02. Data Cleaning and Preprocessing

### Objectives:
1. Handle missing values with PySpark
2. Remove duplicates at scale
3. Fix data type inconsistencies
4. Standardize categorical values
5. Compare performance with Pandas

In [1]:
import sys
import os
import findspark
import yaml
from pyspark.sql import SparkSession

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.append(project_root)

config_path = os.path.join(project_root, 'config', 'pipeline_config.yaml')
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

hadoop_home = os.path.join(project_root, 'bin', 'winutils')
os.environ['HADOOP_HOME'] = hadoop_home
os.environ['PATH'] += os.pathsep + os.path.join(hadoop_home, 'bin')

findspark.init()

spark_memory = config.get('spark', {}).get('memory', {})

spark = SparkSession.builder \
    .appName("Cleaning_PySpark") \
    .config("spark.driver.memory", spark_memory.get('driver', '2g')) \
    .config("spark.executor.memory", spark_memory.get('executor', '2g')) \
    .getOrCreate()

print(f"✅ Spark Session initialized for Cleaning using config memory: {spark_memory.get('driver')}")


✅ Spark Session initialized for Cleaning using config memory: 4g


In [2]:
import time
import os

raw_filename = config['datasets']['superstore']['filename']
raw_path = os.path.join(project_root, 'data', 'raw', raw_filename)

print(f"Loading dataset: {raw_path}")
start_time = time.time()

df = spark.read.csv(
    raw_path,
    header=True,
    inferSchema=True,
    encoding='ISO-8859-1'
)

load_time = time.time() - start_time

print(f"\n📊 Dataset loaded in {load_time:.2f} seconds")
print(f"Rows: {df.count():,}")
print(f"Columns: {len(df.columns)}")

print("\nSchema:")
df.printSchema()

Loading dataset: C:\Users\Administrator\Documents\Luxdev\used-cars-analysis-pipeline\data\raw\superstore_final_dataset.csv

📊 Dataset loaded in 4.01 seconds
Rows: 9,800
Columns: 18

Schema:
root
 |-- Row_ID: integer (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Ship_Date: string (nullable = true)
 |-- Ship_Mode: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal_Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub_Category: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [3]:
from pyspark.sql.functions import col, count, when

print("\n" + "="*70)
print("STEP 1: ANALYZING MISSING VALUES")
print("="*70)

null_counts = df.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in df.columns]
).collect()[0].asDict()

total_rows = df.count()

print("\nMissing values by column:")
for col_name, null_count in sorted(null_counts.items(), key=lambda x: x[1], reverse=True):
    if null_count > 0:
        pct = (null_count / total_rows) * 100
        print(f"  {col_name:<30} {null_count:>8,} ({pct:>6.2f}%)")
    else:
        print(f"  {col_name:<30}        0 (  0.00%)")


STEP 1: ANALYZING MISSING VALUES

Missing values by column:
  Postal_Code                          11 (  0.11%)
  Row_ID                                0 (  0.00%)
  Order_ID                              0 (  0.00%)
  Order_Date                            0 (  0.00%)
  Ship_Date                             0 (  0.00%)
  Ship_Mode                             0 (  0.00%)
  Customer_ID                           0 (  0.00%)
  Customer_Name                         0 (  0.00%)
  Segment                               0 (  0.00%)
  Country                               0 (  0.00%)
  City                                  0 (  0.00%)
  State                                 0 (  0.00%)
  Region                                0 (  0.00%)
  Product_ID                            0 (  0.00%)
  Category                              0 (  0.00%)
  Sub_Category                          0 (  0.00%)
  Product_Name                          0 (  0.00%)
  Sales                                 0 (  0.00%)


In [4]:
# Step 2: Handle Missing Values
print("\n" + "="*70)
print("STEP 2: HANDLING MISSING VALUES")
print("="*70)

# Drop columns with >50% missing
threshold = 0.5
cols_to_drop = []

for col_name, null_count in null_counts.items():
    if (null_count / total_rows) > threshold:
        cols_to_drop.append(col_name)

if cols_to_drop:
    print(f"\n⚠️  Dropping columns with >{threshold*100}% missing: {cols_to_drop}")
    df = df.drop(*cols_to_drop)
else:
    print("\n✅ No columns exceed missing threshold")

# Fill missing values
# For numeric columns: fill with median
# For categorical: fill with mode or 'Unknown'

numeric_cols = [f.name for f in df.schema.fields if str(f.dataType) in ['IntegerType', 'DoubleType', 'FloatType', 'LongType']]
categorical_cols = [f.name for f in df.schema.fields if str(f.dataType) == 'StringType']

print(f"\nFilling missing values:")
print(f"  Numeric columns: {len(numeric_cols)}")
print(f"  Categorical columns: {len(categorical_cols)}")

# Calculate medians for numeric columns
fill_values = {}

for col_name in numeric_cols:
    median_val = df.approxQuantile(col_name, [0.5], 0.01)[0]
    fill_values[col_name] = median_val

# Fill with calculated values
df = df.fillna(fill_values)

# Fill categorical with 'Unknown'
df = df.fillna('Unknown', subset=categorical_cols)

print("\n✅ Missing values handled")


STEP 2: HANDLING MISSING VALUES

✅ No columns exceed missing threshold

Filling missing values:
  Numeric columns: 0
  Categorical columns: 0

✅ Missing values handled


In [5]:
# Step 3: Remove Duplicates
print("\n" + "="*70)
print("STEP 3: REMOVING DUPLICATES")
print("="*70)

rows_before = df.count()
df = df.dropDuplicates()
rows_after = df.count()

duplicates_removed = rows_before - rows_after

print(f"\nRows before: {rows_before:,}")
print(f"Rows after: {rows_after:,}")
print(f"✅ Removed {duplicates_removed:,} duplicate rows")


STEP 3: REMOVING DUPLICATES

Rows before: 9,800
Rows after: 9,800
✅ Removed 0 duplicate rows


In [6]:
from pyspark.sql.functions import col, to_date

print("\n" + "="*70)
print("STEP 4: CONVERTING DATE COLUMNS")
print("="*70)

date_cols = [c for c in df.columns if 'date' in c.lower()]

for col_name in date_cols:
    df = df.withColumn(col_name, to_date(col(col_name), 'd/M/yyyy'))
    print(f"  ✅ Converted {col_name} using format d/M/yyyy")

print("\n✅ Date conversions complete")



STEP 4: CONVERTING DATE COLUMNS
  ✅ Converted Order_Date using format d/M/yyyy
  ✅ Converted Ship_Date using format d/M/yyyy

✅ Date conversions complete


In [7]:
# Step 5: Handle Outliers using IQR method
print("\n" + "="*70)
print("STEP 5: HANDLING OUTLIERS")
print("="*70)

print(f"\nNumeric columns: {numeric_cols}")

for col_name in numeric_cols:
    # Calculate quartiles
    quartiles = df.approxQuantile(col_name, [0.25, 0.75], 0.01)
    Q1, Q3 = quartiles[0], quartiles[1]
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Count outliers
    outliers = df.filter((col(col_name) < lower_bound) | (col(col_name) > upper_bound)).count()
    
    if outliers > 0:
        print(f"\n{col_name}:")
        print(f"  Outliers detected: {outliers:,} ({outliers/rows_after*100:.2f}%)")
        print(f"  Capping to [{lower_bound:.2f}, {upper_bound:.2f}]")
        
        # Cap outliers
        df = df.withColumn(
            col_name,
            when(col(col_name) < lower_bound, lower_bound)
            .when(col(col_name) > upper_bound, upper_bound)
            .otherwise(col(col_name))
        )

print("\n✅ Outlier treatment complete")


STEP 5: HANDLING OUTLIERS

Numeric columns: []

✅ Outlier treatment complete


In [8]:
# Step 6: Standardize Categorical Variables
print("\n" + "="*70)
print("STEP 6: STANDARDIZING CATEGORICAL VARIABLES")
print("="*70)

# Exclude date and ID columns
cat_cols_to_clean = [
    c for c in categorical_cols 
    if not any(x in c.lower() for x in ['date', 'id'])
]

print(f"\nCategorical columns to standardize: {cat_cols_to_clean}")

for col_name in cat_cols_to_clean:
    unique_before = df.select(col_name).distinct().count()
    
    # Trim whitespace and convert to title case
    df = df.withColumn(col_name, initcap(trim(col(col_name))))
    
    unique_after = df.select(col_name).distinct().count()
    
    if unique_before != unique_after:
        print(f"  {col_name}: {unique_before} → {unique_after} unique values")

print("\n✅ Categorical standardization complete")


STEP 6: STANDARDIZING CATEGORICAL VARIABLES

Categorical columns to standardize: []

✅ Categorical standardization complete


In [9]:
# Step 7: Validation Summary
print("\n" + "="*70)
print("CLEANING SUMMARY")
print("="*70)

final_count = df.count()

# Check for remaining nulls
null_check = df.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in df.columns]
).collect()[0].asDict()

total_nulls = sum(null_check.values())

print(f"\n📊 Final Dataset:")
print(f"  Rows: {final_count:,}")
print(f"  Columns: {len(df.columns)}")
print(f"  Missing values: {total_nulls}")
print(f"  Duplicates: 0")

print("\n✅ Data cleaning pipeline complete!")


CLEANING SUMMARY

📊 Final Dataset:
  Rows: 9,800
  Columns: 18
  Missing values: 11
  Duplicates: 0

✅ Data cleaning pipeline complete!


In [10]:
import os

print("\n" + "="*70)
print("SAVING CLEANED DATASET")
print("="*70)

spark.conf.set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")
spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

output_dir = os.path.join(project_root, 'data', 'cleaned')
os.makedirs(output_dir, exist_ok=True)

parquet_path = os.path.join(output_dir, 'superstore_cleaned_spark.parquet')
df.write.mode('overwrite').parquet(parquet_path)
print(f"✅ Saved to: {parquet_path}")

csv_path = os.path.join(output_dir, 'superstore_cleaned_spark.csv')
df.coalesce(1).write.mode('overwrite').option('header', 'true').csv(csv_path)
print(f"✅ Saved to: {csv_path}")

print("\n📁 Cleaned data saved in multiple formats")


SAVING CLEANED DATASET
✅ Saved to: C:\Users\Administrator\Documents\Luxdev\used-cars-analysis-pipeline\data\cleaned\superstore_cleaned_spark.parquet
✅ Saved to: C:\Users\Administrator\Documents\Luxdev\used-cars-analysis-pipeline\data\cleaned\superstore_cleaned_spark.csv

📁 Cleaned data saved in multiple formats


In [11]:
# Show sample of cleaned data
print("\n📋 Sample of cleaned data:")
df.show(10, truncate=False)


📋 Sample of cleaned data:
+------+--------------+----------+----------+--------------+-----------+-----------------+-----------+-------------+-------------+--------------+-----------+-------+---------------+---------------+------------+---------------------------------------------------------------------+-------+
|Row_ID|Order_ID      |Order_Date|Ship_Date |Ship_Mode     |Customer_ID|Customer_Name    |Segment    |Country      |City         |State         |Postal_Code|Region |Product_ID     |Category       |Sub_Category|Product_Name                                                         |Sales  |
+------+--------------+----------+----------+--------------+-----------+-----------------+-----------+-------------+-------------+--------------+-----------+-------+---------------+---------------+------------+---------------------------------------------------------------------+-------+
|67    |US-2016-164175|2016-04-30|2016-05-05|Standard Class|PS-18970   |Paul Stevenson   |Home Office|Unit

In [ ]:
spark.stop()
print("✅ Spark session stopped successfully")


NameError: name 'stop_spark_session' is not defined